<a href="https://colab.research.google.com/github/Brutusa/python-machine-learning/blob/main/classificationCustomNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# By Andujar Brutus
from math import exp
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
import numpy as np

# Import data
cancer = load_breast_cancer()
X = cancer.data
y = cancer.target
cancer_categories = cancer.target_names
cancer_features = cancer.feature_names

# Split data (maitain specific data split)
#X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.70, test_size=.30, random_state=20)

# Split data (randomly organize data split)
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.70, test_size=.30)

# Get data's dimensions
X_train_num_instances = X_train.shape[0]
X_train_num_features = X_train.shape[1]

X_test_num_instances = X_test.shape[0]
X_test_num_features = X_test.shape[1]

y_train_num_instances = y_train.shape[0]

y_test_num_instances = y_test.shape[0]

# Learning rate
learning_rate = 0.05

# Bias Neuron
bias_neuron = 1

# Set starting arrays to all zeroes
parameter_vector = np.zeros((X_train_num_features + 1), dtype=np.float128)
train_instance_factor = np.zeros((X_train_num_instances), dtype=np.float128)
logistic_regression_array = np.zeros((X_train_num_instances), dtype=np.float128)
h_theta_array = np.zeros((X_test_num_instances), dtype=np.float128)
predicted_class = np.zeros((X_test_num_instances), dtype=np.int64)
test_instance_factor = np.zeros((X_test_num_instances), dtype=np.float128)

# Dataset copy for use with bias
X_train_bias = np.zeros((X_train_num_instances, X_train_num_features), dtype=np.float128)
X_test_bias = np.zeros((X_test_num_instances, X_train_num_features), dtype=np.float128)
np.copyto(X_train_bias, X_train)
np.copyto(X_test_bias, X_test)

y_train_bias = np.zeros((y_train_num_instances), dtype=np.float128)
y_test_bias = np.zeros((y_test_num_instances), dtype=np.float128)

np.copyto(y_train_bias, y_train)
np.copyto(y_test_bias, y_test)

# Add bias nueron to front of datasets
X_train_bias = np.insert(X_train_bias, 0, bias_neuron, axis=1)
X_test_bias = np.insert(X_test_bias, 0, bias_neuron, axis=1)

# Get dimensions of new datasets
X_train_bias_num_instances = X_train_bias.shape[0]
X_train_bias_num_features = X_train_bias.shape[1]

X_test_bias_num_instances = X_test_bias.shape[0]
X_test_bias_num_features = X_test_bias.shape[1]

y_train_bias_num_instances = y_train_bias.shape[0]

y_test_bias_num_instances = y_test_bias.shape[0]


# Normalize a full vector
def normalize(v, min, max):
  # Min Max Normalization formula
  new_v = (v - min)/ (max-min)
  return new_v

# Normalize dataset when called
def normalize_vectors():
  # Normalize the training vector
  for instance_X in range(X_train_num_instances):
    # normalize everything from index 1 to end (skipping bias neuron)
    for feature_X in range(1, X_train_num_features + 1):
      X_train_bias[instance_X, feature_X] = normalize(X_train_bias[instance_X, feature_X], np.min(X_train_bias), np.max(X_train_bias))

  # Normalize the testing vector
  for instance_X in range(X_test_num_instances):
     # normalize everything from index 1 to end (skipping bias neuron)
    for feature_X in range(1, X_test_num_features + 1):
      X_test_bias[instance_X, feature_X] = normalize(X_test_bias[instance_X, feature_X], np.min(X_test_bias), np.max(X_test_bias))
  return None

## LOGISTIC REGRESSION ALGORITHM
# Train the model
def train_logistic_regression():
  # For every example in our training dataset
  for instance_X in range(X_train_num_instances):
  # And for every feature
    for feature_X in range(X_train_num_features+1):
      # 1st: Calculate Theta-Transposed X for each instance
      train_instance_factor[instance_X] += (parameter_vector[feature_X] * X_train_bias[instance_X, feature_X])

    # And for every feature after adding up the factor
    for feature_X in range(X_train_num_features+1):
      # 2nd: Calculate gradient descent for every instance and for every feature
      logistic_regression_array[instance_X] = ((1/(1 + (np.exp(-(train_instance_factor[instance_X]))))) - y_train[instance_X]) * X_train_bias[instance_X, feature_X]

      # 3rd: Compute full formula for every feature
      parameter_vector[feature_X] = parameter_vector[feature_X] - (learning_rate*(np.sum(logistic_regression_array[instance_X])))
  return None


# Predict the targets
def test_logistic_regression():
  # For every example in our training dataset
  for instance_X in range(X_test_num_instances):
    # And for every feature
    for feature_X in range(X_test_num_features+1):
     # 1st: Calculate Theta-Transposed X for each instance
      test_instance_factor[instance_X] += (parameter_vector[feature_X] * X_test_bias[instance_X, feature_X])
    # And for every feature
    for feature_X in range(X_test_num_features+1):
      # 2nd: Calculate the target output
      h_theta_array[instance_X] = 1/(1 + (np.exp(-(test_instance_factor[instance_X]))))

      # 3rd: Predict class
      if h_theta_array[instance_X] >= 0.5:
        predicted_class[instance_X] = 1
      else:
        predicted_class[instance_X] = 0
  return None

# Get prediction accuracy for logistic regression
def get_logistic_accuracy():
  correct = 0
  for instance_X in range(X_test_num_instances):
    if predicted_class[instance_X] == y_test[instance_X]:
      correct += 1
  return (correct/len(predicted_class))*100

#-------------------------------------------------------------------------------------

## FEED FORWARD NEURAL NETWORK ##

# 1. Initialize the weights and biases
# Array SHx is the hidden layer array which stores all the weights from inputs x
# b1 bias is within each individual hidden layer array at index 0
b2_bias_weight = (1 - (-1)) * np.random.random_sample() + (-1)

SH1wx = np.zeros((X_train_bias_num_features), dtype=np.float128)
SH2wx = np.zeros((X_train_bias_num_features), dtype=np.float128)
SH3wx = np.zeros((X_train_bias_num_features), dtype=np.float128)
SH4wx = np.zeros((X_train_bias_num_features), dtype=np.float128)
target_output = None

for feature_X in range(X_train_bias_num_features):
  SH1wx[feature_X]= (1 - (-1)) * np.random.random_sample() + (-1)
  SH2wx[feature_X]= (1 - (-1)) * np.random.random_sample() + (-1)
  SH3wx[feature_X]= (1 - (-1)) * np.random.random_sample() + (-1)
  SH4wx[feature_X]= (1 - (-1)) * np.random.random_sample() + (-1)

# SOx is output and wx is the weight from the hidden layer (between -1 and 1)
SO1wx = np.zeros(5, dtype=np.float128)

for hidden_weight_x in range(5):
  SO1wx[hidden_weight_x]= (1 - (-1)) * np.random.random_sample() + (-1)

# 2. Feed the training sample
#Hidden Layer calculations
def SH1_calc(dataset):
  SH1_sum = 0
  for instance_X in range(len(dataset)):
    for feature_X in range(len(dataset[0])):
      SH1_sum += (X_train_bias[instance_X, feature_X] * SH1wx[feature_X])
    return SH1_sum

def SH2_calc(dataset):
  SH2_sum = 0
  for instance_X in range(len(dataset)):
    for feature_X in range(len(dataset[0])):
      SH2_sum += (X_train_bias[instance_X, feature_X] * SH2wx[feature_X])
    return SH2_sum

def SH3_calc(dataset):
  SH3_sum = 0
  for instance_X in range(len(dataset)):
    for feature_X in range(len(dataset[0])):
      SH3_sum += (X_train_bias[instance_X, feature_X] * SH3wx[feature_X])
    return SH3_sum

def SH4_calc(dataset):
  SH4_sum = 0
  for instance_X in range(len(dataset)):
    for feature_X in range(len(dataset[0])):
      SH4_sum += (X_train_bias[instance_X, feature_X] * SH4wx[feature_X])
    return SH4_sum

# 3. Propagate inputs forward. Compute net input & output of each unit in hidden + output layers
# Activation function
def sigmaS(SHx_sum):
  return (1 / (1 + np.exp(-(SHx_sum))))

# Calculated separately for ease of use
def sigmaSH1(dataset):
  return sigmaS(SH1_calc(dataset))

def sigmaSH2(dataset):
  sigmaSH2 = sigmaS(SH2_calc(dataset))
  return sigmaSH2

def sigmaSH3(dataset):
  sigmaSH3 = sigmaS(SH3_calc(dataset))
  return sigmaSH3

def sigmaSH4(dataset):
  sigmaSH4 = sigmaS(SH4_calc(dataset))
  return sigmaSH4

# Output layer
def netO1(dataset):
  return ((SO1wx[0]*bias_neuron) + (SO1wx[1]*sigmaSH1(dataset)) + (SO1wx[2]*sigmaSH2(dataset)) + (SO1wx[3]*sigmaSH3(dataset)) + (SO1wx[4]*sigmaSH4(dataset)))

# Final output layer value
def sigma_netO1(dataset):
  return sigmaS(netO1(dataset))

# set target value
def set_target(target):
  global target_output
  target_output = target

# get target value
def get_target():
  return target_output

# Output Layer Error (Error K)
def output_layer_error(output_result):
  return output_result * (1 - output_result) * (get_target() - output_result)

# Output Layer Error terms
def o1_output_error(dataset):
 return output_layer_error(sigma_netO1(dataset))

def b2_bias_error(dataset):
  return o1_output_error(dataset) * SO1wx[0]

def SH1SO1_error(dataset):
  return o1_output_error(dataset) * SO1wx[1]

def SH2SO1_error(dataset):
  return o1_output_error(dataset) * SO1wx[2]

def SH3SO1_error(dataset):
  return o1_output_error(dataset) * SO1wx[3]

def SH4SO1_error(dataset):
  return o1_output_error(dataset) * SO1wx[4]

# Hidden Layer Errors (Error J)
def hidden_layer_error(SHx_out, output_error, weight):
  error = SHx_out * (1 - SHx_out) * output_error * weight
  return error

def H1_output_error(dataset):
  error = 0
  # bias updates within each array at index 0
  for feature_X in range(X_train_bias_num_features):
    error = hidden_layer_error(sigmaSH1(dataset), o1_output_error(dataset), SH1wx[feature_X])
  return error

def H2_output_error(dataset):
  error = 0
  # bias updates within each array at index 0
  for feature_X in range(X_train_bias_num_features):
    error = hidden_layer_error(sigmaSH2(dataset), o1_output_error(dataset), SH2wx[feature_X])
  return error

def H3_output_error(dataset):
  error = 0
  # bias updates within each array at index 0
  for feature_X in range(X_train_bias_num_features):
    error = hidden_layer_error(sigmaSH3(dataset), o1_output_error(dataset), SH3wx[feature_X])
  return error

def H4_output_error(dataset):
  error = 0
  # bias updates within each array at index 0
  for feature_X in range(X_train_bias_num_features):
    error = hidden_layer_error(sigmaSH4(dataset), o1_output_error(dataset), SH4wx[feature_X])
  return error

# Update weights between hidden and output layer formula
def delta_jk(output_layer_error, hidden_layer_neuron):
  return learning_rate * output_layer_error * hidden_layer_neuron

def update_output_layer_weights(dataset):
  # update weights between hidden and output layers + the bias
  SO1wx[0] += (learning_rate * o1_output_error(dataset))

  SO1wx[1] += delta_jk(SH1SO1_error(dataset), sigmaSH1(dataset))
  SO1wx[2] += delta_jk(SH2SO1_error(dataset), sigmaSH2(dataset))
  SO1wx[3] += delta_jk(SH3SO1_error(dataset), sigmaSH3(dataset))
  SO1wx[4] += delta_jk(SH4SO1_error(dataset), sigmaSH4(dataset))
  return None

# Update weights between input and hidden layer formula
def delta_ij(hidden_layer_error, sigma_SHx):
  return learning_rate * hidden_layer_error * sigma_SHx

def update_hidden_layer_weights(dataset):
  # Update all weights between input and hidden layer + the bias
  for instance_X in range(len(dataset)):
    SH1wx[instance_X] += delta_ij(H1_output_error(dataset), sigmaSH1(dataset))
    SH2wx[instance_X] += delta_ij(H2_output_error(dataset), sigmaSH2(dataset))
    SH3wx[instance_X] += delta_ij(H3_output_error(dataset), sigmaSH3(dataset))
    SH4wx[instance_X] += delta_ij(H4_output_error(dataset), sigmaSH4(dataset))
    return None

# Train ANN
def train_ANN():
  SH1_calc(X_train_bias)
  SH2_calc(X_train_bias)
  SH3_calc(X_train_bias)
  SH4_calc(X_train_bias)

  i = 0
  while( i < len(X_train_bias)):
    set_target(y_train_bias[i])
    update_output_layer_weights(X_train_bias)
    update_hidden_layer_weights(X_train_bias)
    i += 1
  return None

# Test ANN
def test_ANN():
  correct = 0

  for instance_X in range(X_test_bias_num_instances):
    observed_output = 0
    i = 0
    #print("Instance  ", instance_X)
    while(observed_output != y_test_bias[instance_X]):
      set_target(y_test_bias[instance_X])
      # print("epoch ", i)
      # print(SO1wx)
      update_output_layer_weights(X_test_bias)
      update_hidden_layer_weights(X_test_bias)

      if (sigma_netO1(X_test_bias)) >= .5:
        observed_output = 1
      else:
        observed_output = 0
      # used for debugging
      # print("observed output is", observed_output," and target output is ", get_target())
      # print("OUTPUT VALUE IS ", sigma_netO1(X_test_bias))

      i += 1


      if observed_output == get_target():
        correct += 1;

      accuracy = (correct/len(y_test_bias))*100
  return accuracy

normalize_vectors()

train_logistic_regression()
test_logistic_regression()
print("Logistic Regression Accuracy: {:.1f}".format(get_logistic_accuracy()),"%")


# The ANN may take a bit to process the data. please be patient.
train_ANN()
print("Feed Forward ANN Accuracy: {:.1f}".format(test_ANN()),"%")








Logistic Regression Accuracy: 68.4 %
Feed Forward ANN Accuracy: 64.3 %
